## Lifecycle Demo

In [1]:
from IPython.display import display

### 1. Train and log models

In [ ]:
from src.train import train_and_evaluate
from src.mlflow_wrapper import log_run, set_model_alias

MODEL_NAME = "churn-predictor"
CSV_PATH = "data/WA_Fn-UseC_-Telco-Customer-C.csv" # as if: FS.get_features()

result = train_and_evaluate(CSV_PATH, random_state=42)
model_version = log_run(result, MODEL_NAME)
set_model_alias(MODEL_NAME, int(model_version.version)) # Simulate the promotion pipeline step

print("Run ID:", model_version.run_id)
print("Model version:", model_version.version)

/home/ces00/enpal/enpal_2/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2026/02/27 18:48:45 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arb

Run ID: 18ee196876a94e619f535f8c87502f3c
Model version: 2


Registered model 'churn-predictor' already exists. Creating a new version of this model...
Created version '2' of model 'churn-predictor'.


### 2. Load champion and infer

In [7]:
from src.mlflow_wrapper import load_champion

MODEL_NAME = "churn-predictor"

model = load_champion(MODEL_NAME) # simulate deployment step
example = result["X_test"].head(5)
predictions = model.predict(example) # simulate inference step
display(example.assign(predicted_churn=predictions))

,MonthlyCharges,tenure,TotalCharges,HighValueFiber,predicted_churn
2481,25.00,61,1501.75,0,0
6784,24.70,19,465.85,0,0
6125,102.25,13,1359.00,1,1
3052,55.05,37,2030.75,0,0
4099,29.45,6,161.45,0,0


### 3. Show metrics

In [ ]:
from mlflow.tracking.fluent import search_runs
import pandas as pd
from src.mlflow_wrapper import load_config

load_config()
runs = search_runs(output_format="pandas")
assert isinstance(runs, pd.DataFrame)
cols = [c for c in runs.columns if c.startswith("metrics.")]
runs[["run_id", "status"] + cols].head(10)

,run_id,status,metrics.precision,metrics.accuracy,metrics.recall,metrics.f1
0,18ee196876a94e619f535f8c87502f3c,FINISHED,0.55082,0.756219,0.449198,0.494845
1,50887a29492447a8be905f6018976c57,FINISHED,0.55082,0.756219,0.449198,0.494845
